In [39]:
# NOTE: !!! THIS CELL SHOULD BE RUN FIRST !!!

# Jupyter notebooks have a bad interaction with virtual environments: 
# they perceive the PYTHONPATH as the folder the venv was activated
# instead of whatever the current directory of the .ipynb file is
# so we need to incorporate this hack

import sys

filepath = "/Users/colinyao/Desktop/Code/FINM/Portfolio_36700/HW/PortfolioRiskHW/HW3"
# change this path to whatever the path to this file is on your computer
sys.path.append(filepath)

import os
os.chdir(filepath)

In [40]:
import pandas as pd

In [41]:
import numpy as np
import pandas as pd
from scipy.stats import norm

DATA_PATH = "data/spx_returns_weekly.xlsx"   # uploaded file
SHEET_NAME = "s&p500 rets"                        # correct sheet
ALPHA = 0.05
ROLL_WIN_WEEKS = 52                               # rolling vol window
HORIZONS_WEEKS = [1, 4]                           # VaR/CVaR horizons (weeks)
TICKERS_PREFERRED = ["AAPL", "META", "NVDA", "TSLA"]

In [42]:
filepath = "/Users/colinyao/Desktop/Code/FINM/Portfolio_36700/HW/PortfolioRiskHW/HW3/data/spx_returns_weekly.xlsx"
ret_df = pd.read_excel(filepath, sheet_name=2, index_col=0)

In [43]:
instruments = ['AAPL', 'META', 'NVDA', 'TSLA']
sel_df = ret_df[instruments]

In [44]:
# 1.1
def compstat(series, sname):
    variance = series.var()
    empvar = series.quantile(0.05)
    empcvar = series[series <= empvar].mean()
    print(f"Instrument {sname}: Variance {variance}, empirical VaR {empvar}, empirical CVaR {empcvar}")
    return (sname, variance, empvar, empcvar)

csdict = {}
for i in instruments:
    csdict[i] = compstat(sel_df[i], i)
pass

Instrument AAPL: Variance 0.0014716081925336146, empirical VaR -0.056366297517849974, empirical CVaR -0.08312491891904249
Instrument META: Variance 0.0023737852999561686, empirical VaR -0.07001167040022517, empirical CVaR -0.1031960640416827
Instrument NVDA: Variance 0.004127520553944743, empirical VaR -0.08685282018109243, empirical CVaR -0.11645514176034043
Instrument TSLA: Variance 0.006613492943319799, empirical VaR -0.11739730351568925, empirical CVaR -0.14781370062338312


In [45]:
# 1.2
equiweights = pd.Series(sum((1 / len(instruments)) * sel_df[i] for i in instruments), name="equal_weights")
ew_stats = compstat(equiweights, equiweights.name)
pass

Instrument equal_weights: Variance 0.0019147546766932891, empirical VaR -0.06194990495722183, empirical CVaR -0.08499232001174674


# 1.2
We note that the equal-weights portfolio has variance close to the minimum variance of the component instruments. Similarly, empirical VaR and empirical CVaR are close to the minimum empirical VaR and empirical CVaR of the component instruments. This is because the instruments are somewhat diversified, so the variance of their weighted sum is lesser compared to the equivalent sum of their variances, and similarly for VaR and CVaR. 

In [46]:
# 1.3
most_volatile = sorted(((i, sel_df[i].var()) for i in instruments), key=lambda x: x[1], reverse=True)[0][0] # most volatile instrument
instr_dropped = [i for i in instruments if i != most_volatile]
equidropweights = pd.Series(sum((1 / len(instruments)) * sel_df[i] for i in instr_dropped), name="equal_dropped_weights")
ewd_stats = compstat(equidropweights, equidropweights.name)

# comparison
print(f"Difference due to dropping: Variance {ew_stats[1] - ewd_stats[1]}, empirical VaR {ew_stats[2] - ewd_stats[2]}, empirical CVaR {ew_stats[3] - ewd_stats[3]}")

print(f"Stats of the most variant (scaled): Variance {(1 / len(instruments)) * csdict[most_volatile][1]}, empirical VaR {(1 / len(instruments)) * csdict[most_volatile][2]}, empirical CVaR {(1 / len(instruments)) * csdict[most_volatile][3]}")

Instrument equal_dropped_weights: Variance 0.0009169845136893209, empirical VaR -0.042462511580251314, empirical CVaR -0.060531130154127574
Difference due to dropping: Variance 0.0009977701630039682, empirical VaR -0.019487393376970516, empirical CVaR -0.024461189857619164
Stats of the most variant (scaled): Variance 0.0016533732358299498, empirical VaR -0.029349325878922312, empirical CVaR -0.03695342515584578


# 1.3
The risk which the asset adds to the portfolio is as stated in "Difference due to dropping" above. Note that each of the relevant quantities is lesser in magnitude compared to the (scaled) solo asset: this is because some of the variance in the new asset (were it added to the portfolio) is diversified away by the other assets. Similarly, losses as reflected in VaR and CVaR are counterbalanced in some cases by gains in the other assets. 

In [47]:
# ---- load weekly returns ----
rets = pd.read_excel(DATA_PATH, sheet_name=SHEET_NAME, index_col=0, parse_dates=True)

# clean
rets = rets.apply(pd.to_numeric, errors="coerce").dropna(how="all").sort_index()
rets = rets.loc[~rets.index.duplicated(keep="first")]

# choose 4 tickers
available = [t for t in TICKERS_PREFERRED if t in rets.columns]
if len(available) < 4:
    available = list(rets.columns[:4])
tickers = available[:4]

# subset and make equally-weighted portfolio
rets_sel = rets[tickers].dropna(how="any")
w = np.repeat(1/len(tickers), len(tickers))
rp = rets_sel.dot(w)
rp.name = "EW"


In [48]:
# 2.0 HELPERS

def normal_var_cvar(sigma, alpha=0.05, mu=0.0, horizon_weeks=1):
    """
    Normal VaR/CVaR at tail alpha (return units), sqrt(h) scaling.
    VaR is alpha-quantile; CVaR is conditional mean below VaR.
    """
    z = norm.ppf(alpha)
    sig_h = sigma * np.sqrt(horizon_weeks)
    var = mu + sig_h * z
    cvar = mu - sig_h * norm.pdf(z) / alpha
    return var, cvar

def empirical_var_cvar(x, alpha=0.05):
    """ Historical (empirical) VaR & CVaR for a return series x. """
    x = pd.Series(x).dropna()
    var = x.quantile(alpha, interpolation="linear")
    cvar = x[x <= var].mean()
    return var, cvar

In [49]:
# 2.1 CONDITIONAL STATS (END-OF-SAMPLE)

# rolling weekly vol for EW portfolio
roll_std_ew = rp.rolling(window=ROLL_WIN_WEEKS, min_periods=ROLL_WIN_WEEKS).std(ddof=1)

# end-of-sample weekly & annualized vol
sigma_w_end = roll_std_ew.iloc[-1]
ann_factor_weeks = np.sqrt(52)
sigma_ann_end = sigma_w_end * ann_factor_weeks

# normal VaR/CVaR at end using rolling vol (1w & 4w)
rows = []
for h in HORIZONS_WEEKS:
    var_h, cvar_h = normal_var_cvar(sigma=sigma_w_end, alpha=ALPHA, mu=0.0, horizon_weeks=h)
    rows.append({
        "Horizon (weeks)": h,
        "Annualized Vol (as of end)": sigma_ann_end,
        f"Normal VaR {int(ALPHA*100)}%": var_h,
        f"Normal CVaR {int(ALPHA*100)}%": cvar_h
    })
cond_table = pd.DataFrame(rows).set_index("Horizon (weeks)")

# STATIC (full-sample) stats for comparison
sigma_w_full = rp.std(ddof=1)
sigma_ann_full = sigma_w_full * ann_factor_weeks
var_emp, cvar_emp = empirical_var_cvar(rp, alpha=ALPHA)

compare = pd.DataFrame({
    "Weekly Vol (full sample)": [sigma_w_full],
    "Annualized Vol (full sample)": [sigma_ann_full],
    f"Empirical VaR {int(ALPHA*100)}% (full sample)": [var_emp],
    f"Empirical CVaR {int(ALPHA*100)}% (full sample)": [cvar_emp]
}, index=["EW portfolio"])

print("\n--- 2.1: Conditional (end-of-sample) stats for EW portfolio ---")
display(cond_table)

print("\n--- Comparison to static (full-sample) stats from 1.2 ---")
display(compare)


--- 2.1: Conditional (end-of-sample) stats for EW portfolio ---


,Annualized Vol (as of end),Normal VaR 5%,Normal CVaR 5%
Horizon (weeks),,,
1,0.363786,-0.082980,-0.10406
4,0.363786,-0.165959,-0.20812



--- Comparison to static (full-sample) stats from 1.2 ---


,Weekly Vol (full sample),Annualized Vol (full sample),Empirical VaR 5% (full sample),Empirical CVaR 5% (full sample)
EW portfolio,0.043758,0.315543,-0.06195,-0.084992


The stats from 1.2 indicate that the forecast with the normal model estimates a higher annualized vol, higher VaR, and higher CVaR than is found in the sample. 

In [50]:
# 2.2 BACKTEST: HIT RATES (1-week horizon)

alpha = ALPHA
z = norm.ppf(alpha)

# Expanding weekly vol through t
exp_std_ew = rp.expanding(min_periods=20).std(ddof=1)
var_exp_t = z * exp_std_ew                   # VaR_t using data through t
hits_exp = rp.shift(-1) < var_exp_t          # hit if r_{t+1} < VaR_t
hit_rate_exp = hits_exp.dropna().mean()

# Rolling weekly vol (52w window)
var_roll_t = z * roll_std_ew
hits_roll = rp.shift(-1) < var_roll_t
hit_rate_roll = hits_roll.dropna().mean()

bt = pd.DataFrame({
    "Hit Rate (expanding vol)": [hit_rate_exp],
    "Obs (expanding)": [hits_exp.dropna().shape[0]],
    "Hit Rate (rolling vol)": [hit_rate_roll],
    "Obs (rolling)": [hits_roll.dropna().shape[0]],
}, index=["EW portfolio"])

print("\n--- 2.2: VaR hit-test results (alpha = {:.0%}, horizon = 1 week) ---".format(alpha))
display(bt)


--- 2.2: VaR hit-test results (alpha = 5%, horizon = 1 week) ---


,Hit Rate (expanding vol),Obs (expanding),Hit Rate (rolling vol),Obs (rolling)
EW portfolio,0.049815,542,0.02952,542
